# Quickstart Guide: Google Earth Engine

### A little background

Our goal is to create a ***model of Ouagadougou urban hot spots*** as predicted by variables like urban buildup, local vegetation index, elevation above sea level, distance to parks, time of year, etc. 

Many of our variables come from **satellite** and other **earth observation sources**. Before we can do any modeling, we need to acquire this data and preprocess it. This includes, amongst other things:
- cleaning the data (e.g removing outliers and bad quality images or filtering for specific time periods), 
- reprojecting data to the same coordinate system/resolution/timestep for comparability, 
- and often times doing a little numerical magic to ensure our model is mathematically sound (like converting Kelvin to Celsius, computing NDBI from different bands, scaling, averaging, normalizing, or balancing unbalanced features)

### A little tool tip

For our project, we can use **Google Earth Engine (GEE)** with Python (in our Jupyter Notebooks) to download and analyze data. Despite the hurdle of having to learn some Python programming, there are several benefits to using GEE for accessing and processing our satellite data:
- it's free (for research and non-commercial use)
- it offers many modern earth observation products from different official agencies (e.g. Copernicus, NASA)
- a ton of data preprocessing is already done for us
- and it has built-in solutions to process/transform images *in the cloud*, meaning we don't have to use our personal computing resources to download and process data
<br><br>
----

## Initial setup

To get everyone set up, you can follow these general directions.

> [!**NOTE**]
>
> Feel free to reach out to me, if you run into problems or have questions! Even better, let me know what's unclear so we can make this quick start guide even better for other new GEE users.


You only need to do this account + project setup **once** with your Google account. After that, you can just use the same authenticate and initialize commands in any new Jupyter notebook.


## 1. Create your Google Earth Engine account

1. Open this page in your browser:  
   https://earthengine.google.com/signup/
2. Sign in with your google account (the same one you’ll use for this tutorial and all future GEE projects).
3. When asked, choose your account to connect to Earth Engine Code Editor and give it permission to see, edit, manage, etc. the files associated with your account.
4. Agree to the terms and services on the Earth Engine page

After this, your Google account has connected to Earth Engine. Now you just need to make a project to use.

<br>

---

## 2. Create or join a Cloud Project for Earth Engine

Earth Engine runs “through” a Google Cloud project. Every request (like downloading data) is associated with a project, and therefore you need to be linked to a project. 

You have two options:

### **Option A – Use a shared project**  
- If someone else has created a shared project you can use the **project name** they provide.  
  - In *our* case, the project name we’ll use is:  
    `Ouaga Hot Spots`  
1. Make sure you’ve been added to this project (you may receive an email invitation to join; accept it if you see one). If you haven't been invited or need a re-invite, email me.
2. In the top left bar, click the button → **“Select a project”**.
3. A window should pop up with different project names. Select our project called `Ouaga Hot Spots` with the project id `ee-helyne`.


### **Option B – Create your own project (if you’re working independently)**  
1. Create a new project:
    - Go to the Google Cloud Console: https://console.cloud.google.com/earth-engine  
    - In the top left bar, click the button → **“Select a project”**.
    - In the top right of the pop-up, click → **“New project”**.
2. Name the new project & copy the ID:
    - Give it a name (e.g. `Ouaga Hot Spots`) (and optionally change the Project ID)
    - Note down the **Project ID** (a short string like `ouaga-hot-spots`).
    - If you're associated with an Organisation, you can optionally link it to their parent folder, otherwise, select `No organization` for `Location`
    - Click `Create`. 
    - Now you should be able to select the project to connect with you GEE account
2. Register the project for noncommercial use:
    - Go to the Earth Engine registration page:
      `https://code.earthengine.google.com/register?project=ouaga-hot-spots` (or replace `ouaga-hot-spots` with the project ID of the project you just created).
    - Click "Get started" under the section "See if you are eligible for noncommercial use"
    - Follow the prompts and enable the API use.
    - For example, you can use something like this for settings:
      ![GEE Noncommercial use project settings example](gee-settings.png "gee-project-use-settings")


And you're done! Congrats, you now have:
- an **Earth Engine account**, and  
- a **Cloud project ID** (either shared or your own).

That's all you need for authenticating and initializing the project in future notebooks to work with the earth observation data.

# Example

In [4]:
# Import libraries
import ee
import geemap

In [5]:
# Authenticate your session (can be commented out after first authentication)
# this will open up a browser window
# select the right account and cloud project and then click "generate token"
# give GEE access to the requested permissions
# copy the the `Authorization code`
# paste the code into the box below (or above if you're using VS code)
ee.Authenticate()

True

In [6]:
# Initialize the project (change the project ID to your project ID)
ee.Initialize(project="ee-helyne")

In [ ]:
# Example outline of accessing/processing satellite data


# 1. Set study-specific variables (e.g. region of interest)
# Satellites take pictures of the whole world. We only care about Ouagadougou. We need to cut out our area immediately to save processing power.
# Define a point and draw a circle (buffer) around it.
# Ouagadougou Center
roi = ee.Geometry.Point([-1.5197, 12.3714]).buffer(20000)

# 2. Access the data from a specific collection
# We grab Sentinel-2 Surface Reflectance (atmosphere is already corrected)
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# 3. Basic filtering and cleaning
filtered_collection = (
    collection
    .filterBounds(roi)  # Only look at Ouaga
    .filterDate('2022-01-01', '2024-05-30') # Only recent years
    .filter(ee.Filter.calendarRange(3, 5, 'month')) # Only Hot Season (Mar-May)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))    # Only clear images (exclude images that are labelled too cloudy)
)

# 4. Squash the stack into ONE single image
# The median is the middle value of all the images and helps automatically gets rid of bright/dark outlier pixels
image = filtered_collection.median().clip(roi)

# 5. Compute a variable (eg. NDVI, Normalized Difference Vegetation Index)
# Now we have one clean image. But we don't just want "colors," we want "data" (Vegetation, Temperature, etc.).
# NDVI = (NIR - Red) / (NIR + Red)
# In Sentinel-2: NIR is band 'B8', Red is band 'B4'
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

# 6. Visualize
Map = geemap.Map(center=[12.3714, -1.5197], zoom=11)

# Visualization parameters (0 is bare soil, 1 is lush vegetation)
vis_params = {'min': 0, 'max': 0.5, 'palette': ['white', 'green']}

Map.addLayer(ndvi, vis_params, 'Average Vegetation (Hot Season)')
Map

Map(center=[12.3714, -1.5197], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…